In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [ ]:
# Read data with labels
data_labels = pd.read_csv("/kaggle/input/hasyv2-dataset-friend-of-mnist/HASYv2/hasy-data-labels.csv")

In [ ]:
# Display top 5 records
data_labels.head()

In [ ]:
# Dsiplay all unique labels supported for symbols
data_labels["latex"].unique().shape

In [ ]:
# Extracting data only for selected labels
# import string
# cand = ['0','1','2','3','4','5','6','7','8','9','x','+','-','/'] + list(string.printable)

# df = data_labels.loc[data_labels["latex"].isin(cand) ]

In [ ]:
# use the fullset of data
df = data_labels

In [ ]:
# Shape of numpy array after slicing of numpy array based on labels
df.shape

In [ ]:
# Images per labels
df.latex.value_counts().plot.bar()

In [ ]:
# Sample of dataframe after selecting data for requirted labels
df.head()

In [ ]:
# Preparing training data as numpy array of vectors from raw images
import numpy as np
from PIL import Image
from skimage import io, color
from tqdm import tqdm_notebook as tqdm

base = "/kaggle/input/hasyv2-dataset-friend-of-mnist/HASYv2/"
listOfB = []
for items in tqdm(df['path'].iteritems(), total=df["path"].shape[0]): 
    pil_im = Image.open(base +items[1], 'r')
    array1 = np.asarray(pil_im)/255.
    listOfB.append(array1)
B = np.array(listOfB) 
B.reshape(df.shape[0], 32, 32, 3)
B.shape

In [ ]:
train_x = B

In [ ]:
# an example of the training data

import matplotlib.pyplot as plt
plt.imshow(train_x[-789], interpolation='nearest')

In [ ]:
# Indexing raw labels to indexes of lists
print(df['latex'].nunique())
classes = df["latex"].unique().tolist()
n_classes = len(classes)

In [ ]:
# save the labels 
with open("./labels.txt", "w") as f:
    f.write("\n".join(classes))
    
!head ./labels.txt -n 10

In [ ]:
# Preparing categorical labels for multiclass classification (categorical crossentropy)
train_y = df['latex']
listOfk = []
for items in df['latex'].iteritems():
    intermediate = np.zeros(n_classes)
    index = classes.index(items[1])
    intermediate[index] = 1
    listOfk.append(intermediate)
categorical_labels = np.array(listOfk)    
print(categorical_labels.shape)    

In [ ]:
train_y = categorical_labels


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# preparing neural network model using keras API
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(32,32,3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Data generator add randomness to existing data.
#data_generator = keras.preprocessing.image.ImageDataGenerator(validation_split=.1)
## consider using this for more variety
data_generator_with_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=.2, 
    width_shift_range=.2,
    height_shift_range=.2, rotation_range=30,
    zoom_range=.2, 
    shear_range=.1,
#     rescale=1./255,
)


In [ ]:
training_data_generator = data_generator_with_aug.flow(train_x, train_y, subset='training')
validation_data_generator = data_generator_with_aug.flow(train_x, train_y, subset='validation')
history = model.fit(training_data_generator,
#                         batch_size=128,
                        epochs=25,   #5 for demo, use 25 for better results
                        validation_data=validation_data_generator, 
                        callbacks=[EarlyStopping(monitor="val_acc", min_delta=0.1, patience=5)]
                    )

In [ ]:
model.save('./maths_model.h5')

In [ ]:
!ls